In [202]:
import pandas as pd
import numpy as np

In [203]:
url = 'https://raw.githubusercontent.com/lenhattung/SP26-AI1904-DAP391m/refs/heads/main/lab_part_2/01%20-%20Linear%20and%20logistic%20regression/titanic.csv'
df = pd.read_csv(url)

In [204]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   passenger_id  1309 non-null   int64  
 1   pclass        1309 non-null   int64  
 2   survived      1309 non-null   int64  
 3   name          1309 non-null   object 
 4   sex           1309 non-null   int64  
 5   age           1046 non-null   float64
 6   sibsp         1309 non-null   int64  
 7   parch         1309 non-null   int64  
 8   ticket        1309 non-null   object 
 9   fare          1308 non-null   float64
 10  cabin         295 non-null    object 
 11  embarked      1307 non-null   object 
 12  boat          486 non-null    object 
 13  body          121 non-null    float64
 14  home.dest     745 non-null    object 
dtypes: float64(3), int64(6), object(6)
memory usage: 153.5+ KB


,passenger_id,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,1,"Allen, Miss. Elisabeth Walton",1,29.0000,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,2,1,1,"Allison, Master. Hudson Trevor",0,0.9167,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,3,1,0,"Allison, Miss. Helen Loraine",1,2.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,4,1,0,"Allison, Mr. Hudson Joshua Creighton",0,30.0000,1,2,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,5,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",1,25.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"


In [205]:
df['age'] = df['age'].fillna(df['age'].median()).round(0).astype('int64')
df['fare'] = df['fare'].fillna(df['fare'].mean()).round(2).astype('float64')
X = df.drop(['cabin', 'embarked','boat', 'body', 'home.dest', 'name', 'ticket', 'passenger_id','survived'], axis = 1)
y = df['survived']
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   passenger_id  1309 non-null   int64  
 1   pclass        1309 non-null   int64  
 2   survived      1309 non-null   int64  
 3   name          1309 non-null   object 
 4   sex           1309 non-null   int64  
 5   age           1309 non-null   int64  
 6   sibsp         1309 non-null   int64  
 7   parch         1309 non-null   int64  
 8   ticket        1309 non-null   object 
 9   fare          1309 non-null   float64
 10  cabin         295 non-null    object 
 11  embarked      1307 non-null   object 
 12  boat          486 non-null    object 
 13  body          121 non-null    float64
 14  home.dest     745 non-null    object 
dtypes: float64(2), int64(7), object(6)
memory usage: 153.5+ KB


In [206]:
X = sm.add_constant(X)
model_sm = sm.OLS(y, X)
model_sm.fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               survived   R-squared:                       0.365
Model:                            OLS   Adj. R-squared:                  0.362
Method:                 Least Squares   F-statistic:                     124.8
Date:                Fri, 16 Jan 2026   Prob (F-statistic):          1.05e-124
Time:                        23:09:41   Log-Likelihood:                -615.22
No. Observations:                1309   AIC:                             1244.
Df Residuals:                    1302   BIC:                             1281.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.7527      0.059     12.709      0.000       0.636       0.869
pclass        -0.1615      0.017     -9.688      0.000      -0.194      -0.129
sex            0.4979      0.023     21.364      0.000       0.452       0.544
age           -0.0056      0.001     -6.125      0.000      -0.007      -0.004
sibsp         -0.0450      0.011     -3.958      0.000      -0.067      -0.023
parch         -0.0050      0.014     -0.357      0.721      -0.032       0.022
fare           0.0004      0.000      1.488      0.137      -0.000       0.001
==============================================================================
Omnibus:                       54.742   Durbin-Watson:                   1.801
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               61.071
Skew:                           0.529   Prob(JB):                     5.48e-14
Kurtosis:                       2.986   Cond. No.                         368.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [207]:
# Không dùng thư viện cross val score mà code chay để hiểu bản chất
# Dung strafield
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
X.drop('const', axis = 1, inplace = True)
kf = KFold(n_splits = 5, shuffle = True, random_state = 42)
fold_score = []
print(f"{'Fold':<5} | {'Train Size':<10} | {'Val Size':<10} | {'Acc':<10}")
print("-" * 45)

for i, (train_index, val_index) in enumerate(kf.split(X, y)) :
    X_train, X_val = X.iloc[train_index], X.iloc[val_index]
    y_train, y_val = y.iloc[train_index], y.iloc[val_index]
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_val = scaler.transform(X_val)
    model = RandomForestClassifier()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    score = accuracy_score(y_val, y_pred)
    fold_score.append(score)
    print(f"{i+1:<5} | {len(train_index):<10} | {len(val_index):<10} | {score:.4f}")

print("-" * 45)
print(f"Average Accuracy: {np.mean(fold_score):.4f}")
for i, j in zip(y_val, y_pred) :
    print(f'actual : {i} predict : {j}')
kf_a = np.array(fold_score)

Fold  | Train Size | Val Size   | Acc       
---------------------------------------------
1     | 1047       | 262        | 0.7748
2     | 1047       | 262        | 0.7901
3     | 1047       | 262        | 0.7977
4     | 1047       | 262        | 0.7977
5     | 1048       | 261        | 0.7510
---------------------------------------------
Average Accuracy: 0.7823
actual : 1 predict : 1
actual : 1 predict : 1
actual : 1 predict : 1
actual : 1 predict : 0
actual : 0 predict : 0
actual : 1 predict : 0
actual : 1 predict : 1
actual : 1 predict : 1
actual : 0 predict : 0
actual : 1 predict : 0
actual : 0 predict : 1
actual : 0 predict : 0
actual : 1 predict : 0
actual : 1 predict : 1
actual : 1 predict : 0
actual : 1 predict : 1
actual : 1 predict : 1
actual : 1 predict : 1
actual : 1 predict : 1
actual : 1 predict : 1
actual : 1 predict : 1
actual : 1 predict : 1
actual : 1 predict : 1
actual : 1 predict : 0
actual : 1 predict : 1
actual : 0 predict : 0
actual : 1 predict : 0
actual : 1 p

In [208]:
# Sử dụng thư viện cross val score cho nhanh
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier 
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
kf = RepeatedKFold(n_splits = 5, n_repeats=10, random_state = 42)
model = RandomForestClassifier()
score = cross_val_score(model, X, y, cv = kf, scoring = 'accuracy')
print(score)
print(score.mean())
rkf_a = score 
# rkf do chinh xac cao hon

[0.77099237 0.79007634 0.79770992 0.80916031 0.7394636  0.77480916
 0.7480916  0.79770992 0.82442748 0.76245211 0.8129771  0.79007634
 0.79389313 0.77480916 0.79310345 0.83969466 0.76335878 0.78244275
 0.74045802 0.80842912 0.74427481 0.74045802 0.79389313 0.79389313
 0.80842912 0.77862595 0.78625954 0.79389313 0.77862595 0.81992337
 0.80916031 0.80534351 0.74045802 0.79770992 0.7394636  0.75572519
 0.77099237 0.78625954 0.82442748 0.77394636 0.76717557 0.76335878
 0.80152672 0.77862595 0.82375479 0.7480916  0.79770992 0.76717557
 0.77099237 0.80076628]
0.7835015062443333


In [209]:
# Sử dụng thư viện cross val score cho nhanh
# acuracy tang khi du lieu duoc can bang hon (hen xui, co the giam neu du lieu thuc su khong co y nghia)
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier 
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
kf = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 42)
model = RandomForestClassifier()
score = cross_val_score(model, X, y, cv = kf, scoring = 'accuracy')
print(score)
print(score.mean())
skf_a = score

[0.75572519 0.82061069 0.79007634 0.80916031 0.77777778]
0.7906700593723495


In [210]:
# Sử dụng thư viện cross val score cho nhanh
# acuracy tang khi du lieu duoc can bang hon (hen xui, co the giam neu du lieu thuc su khong co y nghia)
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier 
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
kf = RepeatedStratifiedKFold(n_splits = 5, n_repeats=10, random_state = 42)
model = RandomForestClassifier()
score = cross_val_score(model, X, y, cv = kf, scoring = 'accuracy')
print(score)
print(score.mean())
rskf_a = score

[0.75572519 0.83206107 0.77099237 0.81679389 0.8045977  0.80152672
 0.79007634 0.80152672 0.7519084  0.80076628 0.77480916 0.74045802
 0.85877863 0.77099237 0.8045977  0.79007634 0.8129771  0.76717557
 0.76717557 0.81226054 0.79007634 0.79389313 0.74427481 0.82442748
 0.79310345 0.77480916 0.80152672 0.75954198 0.78625954 0.78544061
 0.77480916 0.78244275 0.77099237 0.75954198 0.79310345 0.83587786
 0.79389313 0.80534351 0.76335878 0.75862069 0.77099237 0.78244275
 0.80534351 0.78625954 0.77394636 0.77099237 0.77862595 0.80534351
 0.80916031 0.8045977 ]
0.7880863385101342


In [ ]:
# ket qua cua cac lan cross validation
# du lieu nay thuc su co nghia nen khi chay rskf hay skf thi du lieu van khong bi giam do chinh xac
print(f'kf : {kf_a.mean()}')
print(f'rkf : {rkf_a.mean()}')
print(f'skf : {skf_a.mean()}')
print(f'rskf : {rskf_a.mean()}')

kf : 0.7822526395835162
rkf : 0.7835015062443333
skf : 0.7906700593723495
rskf : 0.7880863385101342


In [213]:
from sklearn.tree import DecisionTreeClassifier

# Train thử 1 cây quyết định
dt = DecisionTreeClassifier()
dt.fit(X, y)

# Xem độ quan trọng của từng cột
import pandas as pd
importance = pd.DataFrame({
    'Feature': X.columns,
    'Score': dt.feature_importances_
}).sort_values(by='Score', ascending=False)

print("--- KIỂM TRA ĐỘ QUAN TRỌNG ---")
print(importance.head(5))

--- KIỂM TRA ĐỘ QUAN TRỌNG ---
  Feature     Score
1     sex  0.308789
5    fare  0.296605
2     age  0.223311
0  pclass  0.109054
3   sibsp  0.045019


In [214]:
from sklearn.model_selection import RepeatedStratifiedKFold, cross_validate # <--- Nhớ import thêm cái này
from sklearn.ensemble import RandomForestClassifier 

# Giả sử X, y đã có sẵn
# Khởi tạo cross-validation
rskf = RepeatedStratifiedKFold(n_splits=5, n_repeats=10, random_state=42)
model = RandomForestClassifier(random_state=42) # Nên để random_state cho model để đồng nhất

# --- PHẦN SỬA ĐỔI QUAN TRỌNG ---
# Dùng cross_validate thay vì cross_val_score
# return_train_score=True: Bắt buộc để lấy điểm tập huấn luyện
results = cross_validate(model, X, y, cv=rskf, scoring='accuracy', return_train_score=True)

# Lấy kết quả ra
train_scores = results['train_score'] # Điểm trên tập học
test_scores = results['test_score']   # Điểm trên tập kiểm thử (Validation)

# Tính trung bình
mean_train = train_scores.mean()
mean_test = test_scores.mean()

# Tính độ Overfit (Gap)
gap = mean_train - mean_test

print(f"Mean Train Score: {mean_train:.4f}")
print(f"Mean Test Score:  {mean_test:.4f}")
print(f"Độ lệch (Gap):    {gap:.4f}")

if gap > 0.1: # Ngưỡng ví dụ, tùy bài toán
    print("=> Kết luận: Có dấu hiệu OVERFIT (Điểm học cao nhưng thi thấp)")
else:
    print("=> Kết luận: Model ổn định (Good Fit hoặc Underfit tùy vào điểm số cao hay thấp)")

Mean Train Score: 0.9661
Mean Test Score:  0.7859
Độ lệch (Gap):    0.1802
=> Kết luận: Có dấu hiệu OVERFIT (Điểm học cao nhưng thi thấp)
